In [2]:
from pathlib import Path
import pandas as pd

In [3]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path(Path.cwd().parent.parent, 'data')

def load_and_combine_data(data_dir, sample_folder='8451_sample', hhd_filename='400_households.parquet', pd_filename='400_products.parquet', tr_filename='400_transactions.parquet'):
    """
    Load household, product, and transaction data from Parquet files, merge them,
    and process date columns.

    Args:
    - data_dir (str or Path): The base directory where data is stored.
    - sample_folder (str): The folder name containing the sample data.
    - households_file (str): Filename of the households data.
    - products_file (str): Filename of the products data.
    - transactions_file (str): Filename of the transactions data.

    Returns:
    - DataFrame: A combined dataframe with processed data.
    """
    # Build file paths
    base_path = Path(data_dir, sample_folder)
    hhd_path = Path(base_path, hhd_filename)
    pd_path = Path(base_path, pd_filename)
    tr_path = Path(base_path, tr_filename)

    # Load dataframes with trimmed column names
    hhd_df = pd.read_parquet(hhd_path).rename(columns=lambda x: x.strip())
    pd_df = pd.read_parquet(pd_path).rename(columns=lambda x: x.strip())
    tr_df = pd.read_parquet(tr_path).rename(columns=lambda x: x.strip())

    # Merge dataframes
    combined_df = tr_df.merge(pd_df, on='PRODUCT_NUM').merge(hhd_df, on='HSHD_NUM')

    # Process date columns
    combined_df['MONTH'] = pd.to_datetime(combined_df['PURCHASE_'], format='%d-%b-%y').dt.month_name()
    combined_df['DATE'] = pd.to_datetime(combined_df['PURCHASE_'], format='%d-%b-%y')

    return combined_df


def get_data():
    return load_and_combine_data(data_dir=DATA_DIR)

In [4]:
df = get_data()

In [5]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Calculate total spend per household
df['TOTAL_SPEND'] = df.groupby('HSHD_NUM')['SPEND'].transform('sum')

# Clean HH_SIZE column
df['HH_SIZE'] = df['HH_SIZE'].str.strip().replace("null", None)
df.dropna(subset=['HH_SIZE'], inplace=True)

# Calculate average spend per household size
average_spend_per_hh_size = df.groupby('HH_SIZE')['TOTAL_SPEND'].mean().reset_index()

# Perform linear regression
X = np.arange(len(average_spend_per_hh_size)).reshape(-1, 1)
y = average_spend_per_hh_size['TOTAL_SPEND'].values.reshape(-1, 1)
regressor = LinearRegression()
regressor.fit(X, y)
y_pred = regressor.predict(X)

# Calculate R-squared
r_squared = r2_score(y, y_pred)

# Plot using Plotly
fig = px.bar(average_spend_per_hh_size, x='HH_SIZE', y='TOTAL_SPEND',
             labels={'HH_SIZE': 'Household Size', 'TOTAL_SPEND': 'Average Spend'},
             title='Average Spend per Household Size')

# Add linear regression line
# Add linear regression line without label
fig.add_trace(go.Scatter(x=average_spend_per_hh_size['HH_SIZE'], y=y_pred.flatten()))
# Add equation and R-squared value to the plot
equation = f'y = {regressor.coef_[0][0]:.2f}x + {regressor.intercept_[0]:.2f}'
r_squared_text = f'R-squared: {r_squared:.2f}'
fig.add_annotation(x=0.05, y=0.95, xref='paper', yref='paper', showarrow=False,
                   text=equation, font=dict(size=12), align='left')
fig.add_annotation(x=0.05, y=0.90, xref='paper', yref='paper', showarrow=False,
                   text=r_squared_text, font=dict(size=12), align='left')

fig.update_layout(showlegend=False)
# Save the plot as HTML file
fig.write_html("plots/average_spend_per_household_size.html")

# Show the plot
fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'plots\\average_spend_per_household_size.html'

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Clean 'INCOME_RANGE' column
df['INCOME_RANGE'] = df['INCOME_RANGE'].str.strip().replace("null", None)
df.dropna(subset=['INCOME_RANGE'], inplace=True)

# Define the desired order of income ranges
income_order = ['UNDER 35K', '35-49K', '50-74K', '75-99K', '100-150K', '150K+']

# Convert 'INCOME_RANGE' to ordered categorical type
income_cat_type = pd.CategoricalDtype(categories=income_order, ordered=True)
df['INCOME_RANGE'] = df['INCOME_RANGE'].astype(income_cat_type)

# Calculate average spend per income range
average_spend_per_income_range = df.groupby('INCOME_RANGE')['TOTAL_SPEND'].mean().reset_index()
average_spend_per_income_range.rename(columns={"TOTAL_SPEND": "AVERAGE_SPEND"}, inplace=True)

# Perform linear regression
X = np.arange(len(income_order)).reshape(-1, 1)
y = average_spend_per_income_range['AVERAGE_SPEND'].values.reshape(-1, 1)
regressor = LinearRegression()
regressor.fit(X, y)
y_pred = regressor.predict(X)

# Calculate R-squared
r_squared = r2_score(y, y_pred)

# Plot using Plotly
fig = px.bar(average_spend_per_income_range, x='INCOME_RANGE', y='AVERAGE_SPEND',
             labels={'INCOME_RANGE': 'Income Range', 'AVERAGE_SPEND': 'Average Spend'},
             title='Average Spend per Income Range',
             category_orders={"INCOME_RANGE": income_order})

# Add linear regression line
fig.add_trace(go.Scatter(x=average_spend_per_income_range['INCOME_RANGE'], y=y_pred.flatten(),
                         mode='lines', name='Linear Regression'))

# Add equation and R-squared value to the plot
equation = f'y = {regressor.coef_[0][0]:.2f}x + {regressor.intercept_[0]:.2f}'
r_squared_text = f'R-squared: {r_squared:.2f}'
fig.add_annotation(x=0.05, y=0.95, xref='paper', yref='paper', showarrow=False,
                   text=equation, font=dict(size=12), align='left')
fig.add_annotation(x=0.05, y=0.90, xref='paper', yref='paper', showarrow=False,
                   text=r_squared_text, font=dict(size=12), align='left')

fig.update_layout(showlegend=False)
# Save the plot as HTML file
fig.write_html("plots/average_spend_per_income_range.html")

# Show the plot
fig.show()


C:\Users\aseem\AppData\Local\Temp\ipykernel_30280\2158019363.py:19: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Clean 'AGE_RANGE' column
df['AGE_RANGE'] = df['AGE_RANGE'].str.strip().replace("null", None)
df.dropna(subset=['AGE_RANGE'], inplace=True)

# Define the desired order of age ranges
age_order = ['19-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75+']

# Convert 'AGE_RANGE' to ordered categorical type
age_cat_type = pd.CategoricalDtype(categories=age_order, ordered=True)
df['AGE_RANGE'] = df['AGE_RANGE'].astype(age_cat_type)

# Calculate average spend per age range
average_spend_per_age_range = df.groupby('AGE_RANGE')['TOTAL_SPEND'].mean().reset_index()

# Perform linear regression
X = np.arange(len(age_order)).reshape(-1, 1)
y = average_spend_per_age_range['TOTAL_SPEND'].values.reshape(-1, 1)
regressor = LinearRegression()
regressor.fit(X, y)
y_pred = regressor.predict(X)

# Calculate R-squared
r_squared = r2_score(y, y_pred)

# Plot using Plotly
fig = px.bar(average_spend_per_age_range, x='AGE_RANGE', y='TOTAL_SPEND',
             labels={'AGE_RANGE': 'Age Range', 'TOTAL_SPEND': 'Average Spend'},
             title='Average Spend per Age Range',
             category_orders={"AGE_RANGE": age_order})

# Add linear regression line
fig.add_trace(go.Scatter(x=average_spend_per_age_range['AGE_RANGE'], y=y_pred.flatten(),
                         mode='lines', name='Linear Regression'))

# Add equation and R-squared value to the plot
equation = f'y = {regressor.coef_[0][0]:.2f}x + {regressor.intercept_[0]:.2f}'
r_squared_text = f'R-squared: {r_squared:.2f}'
fig.add_annotation(x=0.95, y=0.95, xref='paper', yref='paper', showarrow=False,
                   text=equation, font=dict(size=14), align='right')
fig.add_annotation(x=0.95, y=0.90, xref='paper', yref='paper', showarrow=False,
                   text=r_squared_text, font=dict(size=14), align='right')
fig.update_layout(showlegend=False)
# Save the plot as HTML file
fig.write_html("plots/average_spend_per_age_range.html")

# Show the plot
fig.show()

C:\Users\aseem\AppData\Local\Temp\ipykernel_30280\1621292929.py:19: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


# Clean 'CHILDREN' column
df['CHILDREN'] = df['CHILDREN'].str.strip().replace({'': 0, '3+': 3}).fillna(0).astype(int)
df.dropna(subset=['CHILDREN'], inplace=True)

# Calculate average spend per number of children
average_spend_per_children = df.groupby('CHILDREN')['TOTAL_SPEND'].mean().reset_index()

# Perform linear regression
X = average_spend_per_children['CHILDREN'].values.reshape(-1, 1)
y = average_spend_per_children['TOTAL_SPEND'].values.reshape(-1, 1)
regressor = LinearRegression()
regressor.fit(X, y)
y_pred = regressor.predict(X)

# Calculate R-squared
r_squared = r2_score(y, y_pred)

# Plot using Plotly
fig = px.bar(average_spend_per_children, x='CHILDREN', y='TOTAL_SPEND',
             labels={'CHILDREN': 'Number of Children', 'TOTAL_SPEND': 'Average Spend'},
             title='Average Spend per Number of Children')

# Add linear regression line
fig.add_trace(go.Scatter(x=average_spend_per_children['CHILDREN'], y=y_pred.flatten(),
                         mode='lines', name='Linear Regression'))

# Add equation and R-squared value to the plot
equation = f'y = {regressor.coef_[0][0]:.2f}x + {regressor.intercept_[0]:.2f}'
r_squared_text = f'R-squared: {r_squared:.2f}'
fig.add_annotation(x=0.05, y=0.95, xref='paper', yref='paper', showarrow=False,
                   text=equation, font=dict(size=12), align='left')
fig.add_annotation(x=0.05, y=0.90, xref='paper', yref='paper', showarrow=False,
                   text=r_squared_text, font=dict(size=12), align='left')

fig.update_layout(showlegend=False)
# Save the plot as HTML file
fig.write_html("plots/average_spend_per_children.html")

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

# Assume 'df' is your DataFrame and it's already imported.

# Clean 'STORE_R' column
df['STORE_R'] = df['STORE_R'].str.strip().replace("null", None)
df.dropna(subset=['STORE_R'], inplace=True)

# Calculate average spend per store
average_spend_per_store = df.groupby('STORE_R')['TOTAL_SPEND'].mean().reset_index()

# Encode store names numerically for regression analysis
encoder = LabelEncoder()
encoded_store = encoder.fit_transform(average_spend_per_store['STORE_R'])

# Perform linear regression
X = encoded_store.reshape(-1, 1)
y = average_spend_per_store['TOTAL_SPEND'].values.reshape(-1, 1)
regressor = LinearRegression()
regressor.fit(X, y)
y_pred = regressor.predict(X)

# Calculate R-squared
r_squared = r2_score(y, y_pred)

# Plot using Plotly
fig = px.bar(average_spend_per_store, x='STORE_R', y='TOTAL_SPEND',
             labels={'STORE_R': 'Store', 'TOTAL_SPEND': 'Average Spend'},
             title='Average Spend per Store')


fig.write_html("plots/average_spend_per_store.html")
# Show the plot
fig.show()


In [ ]:
df.columns

Index(['BASKET_NUM', 'HSHD_NUM', 'PURCHASE_', 'PRODUCT_NUM', 'SPEND', 'UNITS',
       'STORE_R', 'WEEK_NUM', 'YEAR', 'DEPARTMENT', 'COMMODITY', 'BRAND_TY',
       'NATURAL_ORGANIC_FLAG', 'L', 'AGE_RANGE', 'MARITAL', 'INCOME_RANGE',
       'HOMEOWNER', 'HSHD_COMPOSITION', 'HH_SIZE', 'CHILDREN', 'MONTH', 'DATE',
       'TOTAL_SPEND'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump, load

x_cols = ['STORE_R', 'AGE_RANGE', 'CHILDREN', 'INCOME_RANGE', 'HH_SIZE', 'HOMEOWNER', 'MARITAL', 'HSHD_COMPOSITION']
y_col = 'TOTAL_SPEND'

X = df[x_cols]
y = df[y_col]

# Define categorical columns for OneHotEncoding
categorical_cols =['STORE_R', 'AGE_RANGE', 'CHILDREN', 'INCOME_RANGE', 'HH_SIZE', 'HOMEOWNER', 'MARITAL', 'HSHD_COMPOSITION']

# Create a column transformer to apply OneHotEncoder to categorical columns
column_transformer = ColumnTransformer([
    ('cat', OneHotEncoder(), categorical_cols)
], remainder='passthrough')

# Create a pipeline with transformation and a RandomForestRegressor
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Fitting the model
#pipeline.fit(X_train, y_train)

# Load the model from the file
pipeline = load('random_forest/random_forest_regressor.joblib')

# Now you can use loaded_model to make predictions
y_pred = pipeline.predict(X_test)



# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
ape = np.abs((y_test - y_pred) / y_test) * 100
mape = np.mean(ape)

print(f'Mean Absolute Percentage Error: {mape:.2f}%')
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

Mean Absolute Percentage Error: 10.40%
Mean Squared Error: 3959860.3680308755
R2 Score: 0.93018708079042


In [ ]:
feature_importances = pipeline.named_steps['regressor'].feature_importances_

feature_names = list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out()) + \
                [col for col in x_cols if col not in categorical_cols]

feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_features = sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)

for feature, importance in sorted_features:
    print(f'{feature}: {importance:.3f}')

AGE_RANGE_35-44: 0.103
CHILDREN_0: 0.079
STORE_R_WEST: 0.052
INCOME_RANGE_UNDER 35K: 0.051
STORE_R_EAST: 0.050
AGE_RANGE_55-64: 0.040
INCOME_RANGE_50-74K: 0.039
INCOME_RANGE_75-99K: 0.037
HOMEOWNER_Renter   : 0.037
STORE_R_SOUTH: 0.036
INCOME_RANGE_150K+: 0.035
HSHD_COMPOSITION_1 Adult and Kids : 0.031
INCOME_RANGE_35-49K: 0.031
HH_SIZE_4: 0.029
HH_SIZE_2: 0.027
MARITAL_Single : 0.027
HOMEOWNER_Homeowner: 0.027
AGE_RANGE_45-54: 0.027
AGE_RANGE_75+: 0.024
AGE_RANGE_25-34: 0.022
CHILDREN_1: 0.019
STORE_R_CENTRAL: 0.017
CHILDREN_2: 0.017
MARITAL_Married: 0.016
HSHD_COMPOSITION_Single Male      : 0.016
HH_SIZE_5+: 0.014
AGE_RANGE_65-74: 0.014
HOMEOWNER_null     : 0.013
HH_SIZE_3: 0.013
INCOME_RANGE_100-150K: 0.013
HSHD_COMPOSITION_Single Female    : 0.013
HSHD_COMPOSITION_2 Adults         : 0.008
HSHD_COMPOSITION_1 Adult          : 0.007
CHILDREN_3: 0.005
HSHD_COMPOSITION_2 Adults and Kids: 0.004
AGE_RANGE_19-24: 0.002
HH_SIZE_1: 0.002
MARITAL_null   : 0.002


In [ ]:
import plotly.graph_objects as go

# Extracting labels and values
labels, values = zip(*sorted_features[:5])

# Creating the bar plot
fig = go.Figure(go.Bar(x=labels, y=values))
fig.update_layout(
    title='Feature Importance',
    xaxis_title='Feature',
    yaxis_title='Importance',
    template='plotly_white'
)
fig.write_html("plots/features.html")
fig.show()


In [ ]:
df

NameError: name 'df' is not defined